# Lab Assignment 8: Data Management Using `pandas`, Part 1
## DS 6001: Practice and Application of Data Science

### Instructions
Please answer the following questions as completely as possible using text, code, and the results of code as needed. Format your answers in a Jupyter notebook. To receive full credit, make sure you address every part of the problem, and make sure your document is formatted in a clean and professional way.

In this lab, you will be working with the [2017 Workplace Health in America survey](https://www.cdc.gov/workplacehealthpromotion/survey/data.html) which was conducted by the Centers for Disease Control and Prevention. According to the survey's [guidence document](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Guidance-Document-for-Use-of-Public-Data-files-508.pdf):

> The Workplace Health in America (WHA) Survey gathered information from a cross-sectional, nationally representative sample of US worksites. The sample was drawn from the Dun & Bradstreet (D&B) database of all private and public employers in the United States with at least 10 employees. Like previous national surveys, the worksite served as the sampling unit rather than the companies or firms to which the worksites belonged. Worksites were selected using a stratified simple random sample (SRS) design, where the primary strata were ten multi-state regions defined by the Centers for Disease Control and Prevention (CDC), plus an additional stratum containing all hospital worksites. 

The data contain over 300 features that report the industry and type of company where the respondents are employed, what kind of health insurance and other health programs are offered, and other characteristics of the workplaces including whether employees are allowed to work from home and the gender and age makeup of the workforce. The data are full of interesting information, but in order to make use of the data a great deal of data manipulation is required first.

## Problem 0
Import the following libraries:

In [1]:
import numpy as np
import pandas as pd
import sidetable
import sqlite3
import warnings
warnings.filterwarnings('ignore')

## Problem 1
The raw data are stored in an ASCII file on the 2017 Workplace Health in America survey [homepage](https://www.cdc.gov/workplacehealthpromotion/survey/data.html). Load the raw data directly into Python without downloading the data onto your harddrive and display a dataframe with only the 14th, 28th, and 102nd rows of the data. [1 point]

In [92]:
# Store web address in variable "url"
url = "https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/WHA_120717.sas7bdat"

# Use read_sas and the stored "url"
data = pd.read_sas(url)

# Check the head()
data.head()

,OC1,OC3,HI1,HI2,HI3,HI4,HRA1,HRA1A,HRA1B,HRA1E,...,WL3_05,E1_09,Suppquex,Id,Region,CDC_Region,Industry,Size,Varstrata,Finalwt_worksite
0,7.0,3.0,2.0,1.0,2.0,1.0,1.0,3.0,4.0,2.0,...,b'PTO',NaN,2.0,b'0000000217',1.0,2.0,7.0,7.0,0.0,47.793941
1,2.0,3.0,2.0,3.0,1.0,1.0,1.0,3.0,3.0,1.0,...,NaN,NaN,1.0,b'0000000326',3.0,7.0,7.0,6.0,0.0,47.793941
2,7.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,97.0,2.0,...,NaN,NaN,1.0,b'0000000399',4.0,8.0,7.0,8.0,0.0,47.793941
3,1.0,2.0,1.0,2.0,1.0,1.0,97.0,96.0,96.0,96.0,...,NaN,NaN,1.0,b'0000000475',5.0,9.0,7.0,4.0,0.0,47.793941
4,2.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,3.0,2.0,...,NaN,NaN,1.0,b'0000000489',2.0,4.0,7.0,4.0,0.0,47.793941


In [133]:
# Check the info()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2843 entries, 0 to 2842
Columns: 301 entries, OC1 to Finalwt_worksite
dtypes: float64(295), object(6)
memory usage: 6.5+ MB


## Problem 2 
The data contain 301 columns. Create a new variable in Python's memory to store a working version of the data. In the working version, delete all of the columns except for the following:

* `Industry`: 7 Industry Categories with NAICS codes

* `Size`: 8 Employee Size Categories

* `OC3` Is your organization for profit, non-profit, government?

* `HI1` In general, do you offer full, partial or no payment of premiums for personal health insurance for full-time employees?

* `HI2` Over the past 12 months, were full-time employees asked to pay a larger proportion, smaller proportion or the same proportion of personal health insurance premiums?

* `HI3`: Does your organization offer personal health insurance for your part-time employees?

* `CP1`: Are there health education programs, which focus on skill development and lifestyle behavior change along with information dissemination and awareness building?

* `WL6`: Allow employees to work from home?

* Every column that begins `WD`, expressing the percentage of employees that have certain characteristics at the firm

[1 point]

In [134]:
# Select basic target columns:
target_cols = ['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6']

In [135]:
# Select other columns that begings with "WD"
other_cols = [col for col in data if col.startswith('WD')]
other_cols

['WD1_1', 'WD1_2', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6', 'WD7']

In [136]:
# Update the list of all desired columns, check the length
target_cols = target_cols + other_cols
len(target_cols)

16

In [195]:
# Select desired columns from the original dataset, store in "working_version", check the length 
working_version = data[target_cols]
len(list(working_version.columns))

16

## Problem 3
The [codebook](https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf) for the WHA data contain short descriptions of the meaning of each of the columns in the data. Use these descriptions to decide on better and more intuitive names for the columns in the working version of the data, and rename the columns accordingly. [1 point]

In [196]:
# Take a look at the original column names
working_version.columns

Index(['Industry', 'Size', 'OC3', 'HI1', 'HI2', 'HI3', 'CP1', 'WL6', 'WD1_1',
       'WD1_2', 'WD2', 'WD3', 'WD4', 'WD5', 'WD6', 'WD7'],
      dtype='object')

In [197]:
# Rename to better and more intuitive column names
new_columns ={'Industry':'industry_categories',
             'Size':'size_categories',
             'OC3':'org_profit_type',
             'HI1':'hlth_insur_type',
             'HI2':'hlth_insur_chng12',
             'HI3':'hlth_insur_partime',
             'CP1':'hlth_edu_progs',
             'WL6':'work_from_home',
             'WD1_1':'%empls_under_30',
             'WD1_2':'%empls_over_60',
             'WD2':'%empls_are_female',
             'WD3':'%empls_are_hourly',
             'WD4':'%empls_other_shift',
             'WD5':'%empls_work_remotely',
             'WD6':'%empls_are_unionized',
             'WD7':'%avg_annual_turnover'}

In [198]:
# Apply the map with better names
working_version = working_version.rename(new_columns, axis=1)

In [199]:
# Check the resultant column names
working_version.columns

Index(['industry_categories', 'size_categories', 'org_profit_type',
       'hlth_insur_type', 'hlth_insur_chng12', 'hlth_insur_partime',
       'hlth_edu_progs', 'work_from_home', '%empls_under_30', '%empls_over_60',
       '%empls_are_female', '%empls_are_hourly', '%empls_other_shift',
       '%empls_work_remotely', '%empls_are_unionized', '%avg_annual_turnover'],
      dtype='object')

## Problem 4
Using the codebook and this [dictionary of NAICS industrial codes](https://www.naics.com/search-naics-codes-by-industry/), place descriptive labels on the categories of the industry column in the working data. [1 point]

In [200]:
# Take a look at the original labels for the categories of the industry column
working_version.industry_categories.unique()

array([7., 1., 2., 3., 4., 5., 6.])

In [201]:
# Remap to more descriptive labels
industry_map = {1: 'Agriculture and Fishing, Mining, Utilities, Construction',
                2: 'Manufacturing, Wholesale, Retail, Transportation and Warehousing', 
                3: 'Entertainment and Recreation, Accommodation and Other Services',
                4: 'Information, Finance, Real Estate, Professional, Scientific, Management',
                5: 'Educational Services, Health Care and Social Assistance', 
                6: 'Public Administration',
                7: 'Hospital worksites (NAICS6 = 622110, 622210, 622310)'}

In [202]:
# Remap industries labels using industry_map
working_version['industry_categories']= working_version['industry_categories'].map(industry_map)

In [203]:
# Check the new labels
working_version['industry_categories'].unique()
working_version.stb.freq(['industry_categories'])

,industry_categories,count,percent,cumulative_count,cumulative_percent
0,"Educational Services, Health Care and Social A...",551,19.380936,551,19.380936
1,"Agriculture and Fishing, Mining, Utilities, Co...",525,18.466409,1076,37.847344
2,"Entertainment and Recreation, Accommodation an...",433,15.230390,1509,53.077735
3,"Information, Finance, Real Estate, Professiona...",429,15.089694,1938,68.167429
4,"Hospital worksites (NAICS6 = 622110, 622210, 6...",338,11.888850,2276,80.056279
5,"Manufacturing, Wholesale, Retail, Transportati...",311,10.939149,2587,90.995427
6,Public Administration,256,9.004573,2843,100.000000


## Problem 5
Using the codebook, recode the "size" column to have three categories: "Small" for workplaces with fewer than 100 employees, "Medium" for workplaces with at least 100 but fewer than 500 employees, and "Large" for companies with at least 500 employees. [Note: Python dataframes have an attribute `.size` that reports the space the dataframe takes up in memory. Don't confuse this attribute with the column named "Size" in the raw data.] [1 point]

In [204]:
# Original size categories
working_version['size_categories'].unique()

array([7., 6., 8., 4., 1., 3., 5., 2.])

Source, page 5:  https://www.cdc.gov/workplacehealthpromotion/data-surveillance/docs/2017-WHA-Datafile-Codebook-508.pdf
Note -- these categories have the following meaning 
- 1 = Size Category 1: 10-24 1175 41.33 1175 41.33  -- **Small**
- 2 = Size Category 2: 25-49 655 23.04 1830 64.37 -- **Small**
- 3 = Size Category 3: 50-99 365 12.84 2195 77.21 -- **Small** is 1, 2, 3
- 4 = Size Category 4: 100-249 263 9.25 2458 86.46 -- **Medium** 
- 5 = Size Category 5: 250-499 131 4.61 2589 91.07 -- **Medium** is 4, 5
- 6 = Size Category 6: 500-749 66 2.32 2655 93.39 -- **Large** 
- 7 = Size Category 7: 750-999 28 0.98 2683 94.37 -- **Large**
- 8 = Size Category 8: 1,000+  -- **Large** is 6, 7, 8

In [205]:
# Changing size categories
size_map = {1: 'Small',
            2: 'Small',
            3: 'Small',
            4: 'Medium',
            5: 'Medium',
            6: 'Large',
            7: 'Large',
            8: 'Large'}

In [206]:
# Remap labels using size_map
working_version['size_categories'] = working_version['size_categories'].map(size_map)

In [207]:
# Check the new size categories
working_version['size_categories'].unique()
working_version['size_categories'].value_counts()

Small     2195
Medium     394
Large      254
Name: size_categories, dtype: int64

In [208]:
working_version.stb.freq(['size_categories'])

,size_categories,count,percent,cumulative_count,cumulative_percent
0,Small,2195,77.207176,2195,77.207176
1,Medium,394,13.858600,2589,91.065776
2,Large,254,8.934224,2843,100.000000


## Problem 6
Use the codebook to write accurate and descriptive labels for each category for each categorical column in the working data. Then apply all of these labels to the data at once. Code "Legitimate Skip", "Don't know", "Refused", and "Blank" as missing values. [2 points]

In [209]:
cat_labels = {
    'org_profit_type':{
        1:'For profit, public', 2:'For profit, private', 3:'Non-profit', 4:'State or local government', 
        5:'Federal government', 6:'Other', 97:'', 98:'', 98:''}, #last 3 for the missing values...
    'hlth_insur_type':{
        1:'Full insurance', 2:'Partial insurance', 3:'No insurance', 97:'', 98:'', 99:''},
    'hlth_insur_chng12':{
        1:'Larger',2:'Smaller', 3:'About the same', 97:'', 98:'', 98:'' },
    'hlth_insur_partime':{
        1:'Yes', 2:'No', 97:'', 98:'', 98:'' },
    'hlth_edu_progs':{
        1:'Yes', 2:'No', 97:'', 98:'', 98:'' },
    'work_from_home':{
        1:'Yes',2:'No', 97:'', 98:'', 98:'' }
}

In [210]:
# Replace categorical labels with descriptive labels and propoer missing values 
working_version = working_version.replace(cat_labels)

## Problem 7
The features that measure the percent of the workforce with a particular characteristic use the codes 997, 998, and 999 to represent "Don't know", "Refusal", and "Blank/Invalid" respectively. Replace these values with missing values for all of the percentage features at the same time. [1 point]

In [211]:
num_labels = {
    '%empls_under_30':{997:'0', 998:'0', 999:'0'}, #missing values on the ratio scale is =0
    '%empls_over_60':{997:'0', 998:'0', 999:'0'},
    '%empls_are_female':{997:'0', 998:'0', 999:'0'},
    '%empls_are_hourly':{997:'0', 998:'0', 999:'0'}, 
    '%empls_other_shift':{997:'0', 998:'0', 999:'0'},
    '%empls_work_remotely':{997:'0', 998:'0', 999:'0'},
    '%empls_are_unionized':{997:'0', 998:'0', 999:'0'},
    '%avg_annual_turnover':{997:'0', 998:'0', 999:'0'}
}

In [212]:
# Replace numeric codes with proper missing values 
working_version = working_version.replace(num_labels)

In [213]:
# Check the resultant dataframe with .head()
working_version.head()

,industry_categories,size_categories,org_profit_type,hlth_insur_type,hlth_insur_chng12,hlth_insur_partime,hlth_edu_progs,work_from_home,%empls_under_30,%empls_over_60,%empls_are_female,%empls_are_hourly,%empls_other_shift,%empls_work_remotely,%empls_are_unionized,%avg_annual_turnover
0,"Hospital worksites (NAICS6 = 622110, 622210, 6...",Large,Non-profit,Partial insurance,Larger,No,Yes,Yes,25.0,20.0,85.0,60.0,40.0,15.0,0.0,22.0
1,"Hospital worksites (NAICS6 = 622110, 622210, 6...",Large,Non-profit,Partial insurance,About the same,Yes,Yes,Yes,0,0,90.0,90.0,0,0,0.0,0
2,"Hospital worksites (NAICS6 = 622110, 622210, 6...",Large,Non-profit,Full insurance,About the same,Yes,Yes,Yes,35.0,4.0,0,0,40.0,15.0,0,0
3,"Hospital worksites (NAICS6 = 622110, 622210, 6...",Medium,"For profit, private",Full insurance,Smaller,Yes,No,No,50.0,15.0,50.0,85.0,75.0,0.0,0.0,0
4,"Hospital worksites (NAICS6 = 622110, 622210, 6...",Medium,Non-profit,Full insurance,About the same,Yes,Yes,Yes,50.0,40.0,60.0,60.0,40.0,30.0,0.0,28.0


## Problem 8
Sort the working data by industry in ascending alphabetical order. Within industry categories, sort the rows by size in ascending alphabetical order. Within groups with the same industry and size, sort by percent of the workforce that is under 30 in descending numeric order. [1 point]

In [214]:
working_version.sort_values(['industry_categories', 'size_categories', '%empls_under_30'], 
                            ascending = [True, True, False])

,industry_categories,size_categories,org_profit_type,hlth_insur_type,hlth_insur_chng12,hlth_insur_partime,hlth_edu_progs,work_from_home,%empls_under_30,%empls_over_60,%empls_are_female,%empls_are_hourly,%empls_other_shift,%empls_work_remotely,%empls_are_unionized,%avg_annual_turnover
941,"Agriculture and Fishing, Mining, Utilities, Co...",Large,"For profit, private",Partial insurance,,,Yes,Yes,0,0,50.0,0,0,0,0.0,0
1238,"Agriculture and Fishing, Mining, Utilities, Co...",Large,"For profit, public",Full insurance,Larger,No,Yes,99.0,0,0,0,0,0,0,0,0
1731,"Agriculture and Fishing, Mining, Utilities, Co...",Large,"For profit, public",Partial insurance,About the same,No,No,Yes,0,0,0,0,0,0,10.0,0
2586,"Agriculture and Fishing, Mining, Utilities, Co...",Large,"For profit, private",Full insurance,Larger,No,No,Yes,0,0,54.0,85.0,0.0,1.0,1.0,45.0
1732,"Agriculture and Fishing, Mining, Utilities, Co...",Large,"For profit, private",Partial insurance,About the same,No,Yes,No,50.0,10.0,50.0,75.0,10.0,0.0,0.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,Public Administration,Small,State or local government,Full insurance,About the same,No,No,No,0.0,10.0,50.0,90.0,70.0,0.0,0.0,10.0
2001,Public Administration,Small,Non-profit,Full insurance,About the same,Yes,No,No,0.0,15.0,95.0,0.0,0.0,25.0,0.0,25.0
2302,Public Administration,Small,State or local government,Partial insurance,Smaller,No,No,No,0.0,46.0,59.0,23.0,23.0,0.0,15.0,2.0
2513,Public Administration,Small,Non-profit,Partial insurance,Larger,Yes,No,Yes,0.0,10.0,65.0,99.0,71.0,12.0,0.0,0


## Problem 9
There is one row in the working data that has a `NaN` value for industry. Delete this row. Use a logical expression, and not the row number. [1 point]

In [215]:
# For some reason I do not see the NaN in the industry column. Probably it was removed when renaming labels?
working_version['industry_categories'].isnull().values.any()

False

## Problem 10
Create a new feature named `gender_balance` that has three categories: "Mostly men" for workplaces with between 0% and 35% female employees, "Balanced" for workplaces with more than 35% and at most 65% female employees, and "Mostly women" for workplaces with more than 65% female employees. [1 point]

In [216]:
# Check for missing values
working_version['%empls_are_female'].isnull().values.any()

False

In [224]:
# Infer "float" object type with .astype()
working_version['%empls_are_female'] = working_version['%empls_are_female'].astype('float')

In [226]:
# Define a mapping fucntion and apply it to '%empls_are_female' column
def balance_groups(x):
    if x < 0.35:
        return 'Mostly men'
    elif x > 0.35:
        return 'Balanced'
    elif x < 0.65:
        return 'Balanced'
    elif x > 0.65:
        return 'Mostly women'
    else:
        return ''
working_version['gender_balace'] = working_version['%empls_are_female'].apply(balance_groups)

In [228]:
# Check results in the new 'gender_balace' column
working_version['gender_balace'].head()

0      Balanced
1      Balanced
2    Mostly men
3      Balanced
4      Balanced
Name: gender_balace, dtype: object

## Problem 11
Change the data type of all categorical features in the working data from "object" to "category". [1 point]

## Problem 12
Filter the data to only those rows that represent small workplaces that allow employees to work from home. Then report how many of these workplaces offer full insurance, partial insurance, and no insurance. Use a function that reports the percent, cumulative count, and cumulative percent in addition to the counts. [1 point]

## Problem 13
Anything that can be done in SQL can be done with `pandas`. The next several questions ask you to write `pandas` code to match a given SQL query. But to check that the SQL query and `pandas` code yield the same result, create a new database wsing the `sqlite3` package and input the cleaned WHA data as a table in this database. (See module 6 for a discussion of SQlite in Python.) [1 point]

## Problem 14
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT size, type, premiums AS insurance, percent_female FROM whpps
WHERE industry = 'Hospitals' AND premium_change='Smaller'
ORDER BY percent_female DESC;
```
For each of these queries, your feature names might be different from the ones listed in the query, depending on the names you chose in problem 3.
[2 points]

## Problem 15
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT industry, 
    AVG(percent_female) as percent_female, 
    AVG(percent_under30) as percent_under30,
    AVG(percent_over60) as percent_over60
FROM whpps
GROUP BY industry
ORDER BY percent_female DESC;
```
[2 points]

## Problem 16
Write `pandas` code that replicates the output of the following SQL code:
```
SELECT gender_balance, premiums, COUNT(*)
FROM whpps
GROUP BY gender_balance, premiums
HAVING gender_balance is NOT NULL and premiums is NOT NULL;
```
[2 points]